# Cluster the Image Embeddings and Visualize

After generating images in notebook 2, we now cluster them based on their feature embeddings. We can then visualize and evaluate the cluster to see what types of images were generated and how well different types of images were clustered together.

*Workshop*: AI/ML Pipeline - Synthetic Data Generation; January 23, 2026  
*Platform*: CyVerse Jupyter Lab PyTorch GPU

## Pipeline Overview

1. Load configuration and source data
2. Use CLIP to get embeddings of the synthetic data
3. Cluster the embeddings and evaluate those clusters
4. Visualize the clusters
5. Review results

In [ ]:
# Just in case
# %pip install umap-learn hdbscan scikit-learn matplotlib seaborn transformers torch pillow
# %pip install plotly nbformat
# #Install the specific version that includes the bundled engine
# %pip install "kaleido==0.2.1"
# or
# pip install plotly[kaleido]

In [ ]:
import sys
import torch
import numpy as np
import pandas as pd
from pathlib import Path

#visualization and density analysis
import umap
import hdbscan
from sklearn.metrics import silhouette_score, calinski_harabasz_score
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.manifold import TSNE
import plotly.io as pio
pio.renderers.default = "notebook"

# Model
from PIL import Image
from tqdm.notebook import tqdm
from huggingface_hub import login
from transformers import CLIPProcessor, CLIPModel

# Device Setup (CLIP is light, so CPU is usually fine, but GPU is faster)
device = "cuda" if torch.cuda.is_available() else "cpu"

### Load Configuration

Review and adjust generation parameters if needed.
Remember to include your HF user token to access the free models
- https://huggingface.co/ > login or signup
    - Billing > Access Tokens

In [ ]:
# Add parent directory to path
parent_dir = Path.cwd().parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

print("bringing in local modules")
# brining in more modeules because of image generation process
# modules are found in the DataCollection/src folder
# from src import config, gemini_client, data_loader, prompt_builder, output_handler
from src import config, data_loader, output_handler

print("All modules imported successfully")
print(f"Working directory: {Path.cwd()}")

# Load configuration
# DataCollection/src/config.py ... def load_confi()
# using "generation_config.yaml" for setup
cfg = config.load_config()
data_dir = cfg.get_data_path('generated')
img_dir = cfg.get_data_path('generated/images')
output_dir = cfg.get_data_path('generated/analysis')
print(img_dir)

# Login to HuggingFace just in case it's needed for clip model
# login(token = 'your token')
# login(token = '')

## Create Image Feature Embeddings
Using the CLIP model

CLIP, or contrastive language-image pretraining, uses a Vision Transformer (ViT) model pre-trained on about 400 million contrastive image text pairs to encode the images. Essentially, the model references learned visual concepts from the natural language pre-training which supports zero-shot transfer for learning on new data (Radford et al, 2021). The CLIP model in this work comes from HuggingFace’s OpenAI and uses these image-text pairs for feature embedding and image classification tasks further down the pipeline. 


In [ ]:
def get_image_embeddings(image_dir):
    
    print("Loading CLIP model")
    # Use the standard OpenAI CLIP model
    model_id = "openai/clip-vit-base-patch32"
    model = CLIPModel.from_pretrained(model_id).to(device)
    processor = CLIPProcessor.from_pretrained(model_id)

    # image_files = sorted(list(image_dir.glob("*.png")))
    image_files = sorted(list(handler.images_dir.glob(f"*.{handler.image_format}")))

    embeddings = []
    filenames = []
    print(f"Processing {len(image_files)} images")
    
    # Process in batches to be memory safe
    batch_size = 32
    for i in tqdm(range(0, len(image_files), batch_size)):
        batch_paths = image_files[i:i + batch_size]
        
        # Load images
        images = [Image.open(p) for p in batch_paths]
        
        # Preprocess
        inputs = processor(images=images, return_tensors="pt", padding=True).to(device)
        
        # Inference (Get features)
        with torch.no_grad():
            outputs = model.get_image_features(**inputs)
        
        # Normalize embeddings (crucial for cosine similarity/clustering)
        outputs = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
        
        # Store as numpy array (move to CPU first)
        embeddings.append(outputs.cpu().numpy())
        filenames.extend([p.name for p in batch_paths])

    # Concatenate all batches
    if embeddings:
        embeddings = np.vstack(embeddings)
        return filenames, embeddings
    else:
        return [], np.array([])
        

In [ ]:
# Based on the output handler module,
# This automatically creates folders in DataCollection/data/generated: images/, metadata/, logs/
handler = output_handler.OutputHandler(
    output_dir=cfg.get_output_path(),  # Uses path from generation_config.yaml
    image_format=cfg.output.get('format', 'png'),
    export_csv=True,
    date_organized=True
)
# Run Extraction
filenames, clip_embeddings = get_image_embeddings(img_dir)
print(f"Extracted Embeddings Shape: {clip_embeddings.shape}") 
# Expected shape: (Num_Images, 512)

## Cluster Embeddings and Evaluate
- UMAP, HDBSCAN, Visualize with t-SNE
UMAP, or Uniform Manifold Approximation and Projection, is a dimensionality reduction technique that visualizes embeddings (image or text) in a high dimensional space. Essentially, it groups clusters in high-dimensional space, then optimizes embeddings in the clusters to a low-dimensional layout while still preserving the feature distances (McInnes, Healy, Melville,  2018). 

The HDBSCAN algorithm (HIerarchical Density Based Spatial Clustering of Applications with Noise), is a density based clustering algorithm that finds clusters of various densities instead of trying to fit uniform clusters. It works well for real-world applications that might have imbalanced data. HDBSCAN is a clustering algorithm that groups features using hierarchical partitioning to maximize the stability of the selected clusters while also accounting for noise (Campello, Moulavi, Sander, 2013).

In [ ]:
def run_clustering_analysis(embeddings, filenames):
    print("Running UMAP dimensionality reduction")
    # Reduce to 15 dimensions for density scanning (standard practice)
    # n_neighbors: controls how much local structure is preserved
    umap_reducer = umap.UMAP(
        n_neighbors=15, 
        n_components=15, 
        metric='cosine', 
        random_state=42
    )
    coords_2d = umap_reducer.fit_transform(embeddings)
    
    print("Running HDBSCAN density clustering")
    # Cluster the reduced data
    # min_cluster_size: smallest group to consider a "cluster"
    # min_samples: how conservative the clustering is
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=3, # Low number because you likely have a small test batch
        min_samples=1, 
        metric='euclidean'
    )
    labels = clusterer.fit_predict(coords_2d)
    probabilities = clusterer.probabilities_
    print(probabilities)

    num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    print(f"Found {num_clusters} clusters (Label -1 represents 'noise' or 'outliers')")
    
    # Filter out Noise (Label -1) for fair scoring
    mask = labels != -1
    clustered_data = coords_2d[mask]
    clustered_labels = labels[mask]
    n_clusters = len(set(clustered_labels))
    
    print("\n" + "="*40)
    print(f"({n_clusters} clusters found)")
    print("="*40)
    
    if n_clusters > 1:
        # Silhouette: Measure of how similar an object is to its own cluster (cohesion) 
        # compared to other clusters (separation). Range: -1 to 1.
        sil_score = silhouette_score(clustered_data, clustered_labels)
        
        # Calinski-Harabasz: Ratio of the sum of between-clusters dispersion and 
        # of within-cluster dispersion. Higher is better.
        ch_score = calinski_harabasz_score(clustered_data, clustered_labels)
        
        print(f"Silhouette Score:       {sil_score:.4f}  (Target: > 0.5)")
        print(f"Calinski-Harabasz:      {ch_score:.4f}  (Higher is better)")
    else:
        print("Not enough clusters (need > 1) to calculate separation scores.")

    # create dataframe
    df = pd.DataFrame({
        'filename': filenames,
        'x': coords_2d[:, 0],
        'y': coords_2d[:, 1],
        'cluster': labels,
        'probability': probabilities
    })
    
    # Sort so 'Cluster 0' is at top, and Noise (-1) is at bottom
    df = df.sort_values(by=['cluster', 'probability'], ascending=[True, False])
    print(df.head())

    # Save the data to CSV so you can inspect which images belong to which cluster
    df.to_csv(output_dir /"cluster_probs.csv")
    
    return df, labels


In [ ]:
# Run Clustering and Execute the Analysis

if len(filenames) > 5:
    # cluster_labels = cluster_embeddings(clip_embeddings)
    df_results, labels = run_clustering_analysis(clip_embeddings, filenames)
    print("\nAnalysis Complete. Data is ready for visualization.")
else:
    print("Not enough images to run clustering (Need > 5). Using placeholder labels.")
    cluster_labels = np.zeros(len(filenames))

## Evaluate Clusters
- Silhouette Score
- Calinski-Harabasz Index 

 The output of these clusters was evaluated using the Silhouette Score, which represents the density and separation of a cluster. Specifically, the silhouette score is a clustering evaluation metric that evaluates the cluster quality of an algorithm, particularly for unsupervised data. It’s used to study the separation distance between the resulting clusters where a score closer to 1 shows that a particular sample is far away from the neighboring cluster, where a value of 0 shows a sample point is close to a decision boundary between neighboring clusters, the score provides an average value for all samples (Learn, 2017).

 The Calinski-Harabasz Index (CH) quantifies the ratio of intracluster and intercluster variance, i.e. the ratio of within and between cluster variance. This score helps indicate how compact and well separated a cluster is from other clusters, to create a good distribution (Rachwal et al, 2023). It divides the variance of the sums of squares of the distances of individual data points to their cluster center by the sum of squares to the distance between other cluster centers (Rachwal et al, 2023). A higher CH index shows that clusters are dense and well separated, but there is no index score upper bound, making the score interpretation relative to other metrics (Rachwal et al, 2023).

In [ ]:
def visualize_clusters(df):
    print("Generating Interactive Graph")
    
    # Convert cluster to string so Plotly treats it as a distinct Category (colors), 
    # rather than a continuous number (gradient)
    df['cluster_str'] = df['cluster'].astype(str)
    
    # Create the Interactive Plot
    fig = px.scatter(
        df, 
        x='x', 
        y='y', 
        color='cluster_str',
        symbol='cluster_str', # Different shapes for different clusters
        hover_data=['filename', 'probability'], # Shows up when you mouse over dots
        title='Interactive Synthetic Image Map (t-SNE/UMAP Projection of CLIP Embeddings)',
        color_discrete_sequence=px.colors.qualitative.G10 # A nice distinct color palette
    )
    
    # Styling for better readability
    fig.update_traces(marker=dict(size=10, line=dict(width=1, color='DarkSlateGrey')))
    fig.update_layout(
        legend_title_text='Cluster ID',
        plot_bgcolor='white',
        xaxis_title="Dimension 1",
        yaxis_title="Dimension 2",
        height=700 # Large height for detail
    )

    # save the visualization
    # You can open this file in any web browser later and still zoom/hover
    html_path = output_dir / "cluster_map_interactive.html"
    fig.write_html(str(html_path))
    png_path = output_dir / "cluster_map_static.png"
    fig.write_image(str(png_path), scale=2)
    
    fig.show()
    

    

In [ ]:
# print(df_results.head())
# Execute the Visualization
visualize_clusters(df_results)

# Display Data Tables
print("\nHigh-Confidence Cluster Assignments:")
# Show the "best" example of each cluster
display(df_results[df_results['cluster'] != -1].groupby('cluster').head(2))

print("\nOutliers / Noise (Cluster -1):")
display(df_results[df_results['cluster'] == -1].head(5))

### If plotly doesn't work!
Use regular matpotlib vizualization

In [ ]:
def visualize_results(embeddings, labels, filenames):
    print("Running t-SNE for visualization")
    
    # Adjust perplexity based on dataset size
    n_samples = len(embeddings)
    perplex = min(30, n_samples - 1) if n_samples > 1 else 1

    tsne = TSNE(
        n_components=2, 
        perplexity=perplex, 
        random_state=42, 
        init='pca', 
        learning_rate='auto'
    )
    tsne_results = tsne.fit_transform(embeddings)
    
    # Create DataFrame for plotting
    df = pd.DataFrame({
        'x': tsne_results[:, 0],
        'y': tsne_results[:, 1],
        'cluster': labels,
        'filename': filenames
    })
    
    # Plot
    plt.figure(figsize=(12, 8))
    
    # Scatter plot with HDBSCAN clusters as colors
    # We use a distinct color palette. Outliers (-1) usually colored grey/black.
    sns.scatterplot(
        data=df, 
        x='x', 
        y='y', 
        hue='cluster', 
        palette='tab10', 
        style='cluster',
        s=100, 
        alpha=0.8
    )
    
    plt.title('Synthetic Image Landscape (t-SNE Projection of CLIP Embeddings)', fontsize=15)
    plt.xlabel('t-SNE Dimension 1')
    plt.ylabel('t-SNE Dimension 2')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title="HDBSCAN Cluster")
    
    # Optional: Label points with filenames (cluttered if too many images)
    if len(df) < 55:
        for i in range(df.shape[0]):
            plt.text(
                df.x[i]+0.2, 
                df.y[i], 
                df.filename[i], 
                fontsize=8, 
                alpha=0.7
            )
            
    # Save Plot
    save_path = output_dir / "tsne_cluster_map.png"
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Visualization saved to {save_path}")
    
    # return df


In [ ]:
# Run clustering Vizualization
visualize_results(clip_embeddings, labels, filenames)

